In [25]:
import sys
sys.path.append('../model/dataset.py')
sys.path.append('..')
from model import dataset
from utils import import_data
images_with_ball, images_without_ball = import_data('../predictions/image10.jpg')
train_dataset = dataset.ImageDataset(
    images_with_ball,
    images_without_ball,
    transforms=None
)

In [27]:
import numpy as np
print(len(train_dataset))
image, label = train_dataset[1100]
image = image.permute(1 , 2 , 0).numpy()
image = image.astype(np.uint8)
print(label)
import cv2
cv2.imshow('img', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1152
tensor(1.)
